# Libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import warnings

from pandas.errors import SettingWithCopyWarning
from Bio.Seq import Seq
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from utils import *

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

%matplotlib inline

# Data import

## Antibody pairs

In [2]:
antibody_pairs = pd.read_csv(os.path.dirname(os.getcwd()) + "/data/antibody_pairs/Antibody_pairs.csv")

## Epitope clone ID mapping

In [3]:
epitope_clone_mapping = pd.read_csv(os.path.dirname(os.getcwd()) + 
    "/data/antibody_pairs/Epitope_PDB_clone_ID.csv")
print("Number of antibodies clustered by IGX-Cluster:", len(epitope_clone_mapping))

Number of antibodies clustered by IGX-Cluster: 54


## Antibody pair sequence dataframes

In [4]:
both_chains = pd.read_csv(os.path.dirname(os.getcwd()) + 
    "/data/antibody_pairs/Antibody_pairs_backtranslated_both_chains.csv")

## IGX-Cluster clusters

In [5]:
cluster_full = pd.read_csv(os.path.dirname(os.getcwd()) + "/data/clustering/IGX_Cluster_clustering.tsv", sep="\t",
    low_memory=False)

In [6]:
# separate germline and study sequences for all dataframes
germline, cluster = separate_germline_seq(cluster_full) 

# filter for heavy and light chains
cluster_heavy = cluster[cluster["Chain"] == "Heavy"]
cluster_light = cluster[cluster["Chain"].isin(["Lambda", "Kappa"])]

Germline sequences: 10197
Unique clones: 10490


## SAAB+ clusters

```
SAAB_PLUS -f ~/clustering-comparison/data/FASTA/Simulated_sequences_heavy_antibody_pairs.fasta -o ~/clustering-comparison/data/SAABplus/SAABplus_clustering_heavy_chains_antibody_pairs.tsv -c H -s human
```

In [7]:
saabplus = pd.read_csv(os.path.dirname(os.getcwd()) + "/data/clustering/SAABplus_clustering_heavy_chains.tsv", 
    sep="\t", index_col=0)
saabplus[:5]

,Protein_Seq,CDR-H3_template,Redundancy,Framework_template,CDR-H3_sequence,ESS,Annotation,H1_Canonical_Class,H2_Canonical_Class,Clusters
0,VESGGGLVQPRGSLRLSCAASGFTVSSNEMSWIRQAPGKGLEWVSS...,4edwH,1,5i1aH,ARYGHIVATSWYFDL,66,True,H1-8-A,NaN,4edwH
1,VQSGAEVKKPGASVKVSCKTSGYTFTDYYIHWVRQAPGQGLEWMGW...,6b08C,1,6azzF,ARDRITTAAPFDY,102,True,H1-8-A,H2-8-A,6b08C
2,QESGPGLVKPSQTLSLTCTVSGGSISSGDYYWSWIRQPPGKGLEWI...,4hdiH,1,5utzH,AREPCGGDCYFQH,37,False,H1-10-A,H2-7-A,4hdiH
3,VQSGAEVKKPGSSVKVSCKASGGTFSSYAISWVRQAPGQGLEWMGG...,NaN,1,4m62H,ARDWPRIMITFGGVIVIPGDYFDY,0,False,H1-8-A,H2-8-A,NaN
4,QQWGAGLLKPSETLSLTCAVYGGSFSGYYWSWIRQPPGKGLEWIGE...,NaN,1,6mlkH,ARGGITMIVVVITTEYFQH,0,False,H1-8-A,H2-7-A,NaN


## SPACE2 clusters

In [8]:
space2 = pd.read_csv(os.path.dirname(os.getcwd()) + "/data/clustering/SPACE2_clustering.csv")
space2.rename(columns={"ID":"Clone_ID"}, inplace=True)
space2[:5]

,Clone_ID,cluster_by_length,cluster_by_rmsd
0,/scistor/informatica/kwy700/clustering-compari...,13_10_17_11_8_11,/scistor/informatica/kwy700/clustering-compari...
1,/scistor/informatica/kwy700/clustering-compari...,13_10_17_11_8_11,/scistor/informatica/kwy700/clustering-compari...
2,/scistor/informatica/kwy700/clustering-compari...,13_10_17_11_8_11,/scistor/informatica/kwy700/clustering-compari...
3,/scistor/informatica/kwy700/clustering-compari...,13_10_17_11_8_11,/scistor/informatica/kwy700/clustering-compari...
4,/scistor/informatica/kwy700/clustering-compari...,13_10_17_11_8_11,/scistor/informatica/kwy700/clustering-compari...


# Preprocessing

## SAAB+

In [9]:
saabplus["Clusters"].value_counts(dropna=False)[:5]

Clusters
NaN      1302
5gguA     143
5utzJ     120
5vagC     111
4kteH     110
Name: count, dtype: int64

In [10]:
saabplus_clustered = saabplus[~saabplus["Clusters"].isna()]
saabplus_unclustered = saabplus[saabplus["Clusters"].isna()]

In [11]:
clones_seq = cluster_heavy[["Unique Clone Id", "Receptor Amino Acids"]]
clones_seq.columns = ["Clone_ID", "Protein_Seq"]

# add clone ID to dataset by merging on protein sequence
saabplus_clustered_clones = clones_seq.merge(saabplus_clustered, on="Protein_Seq")
saabplus_antibody_pairs = saabplus_clustered_clones.merge(epitope_clone_mapping, on="Clone_ID", how="inner")
print("Number of antibodies clustered by SAAB+:", len(saabplus_antibody_pairs))
saabplus_antibody_pairs[:5]

Number of antibodies clustered by SAAB+: 54


,Clone_ID,Protein_Seq,CDR-H3_template,Redundancy,Framework_template,CDR-H3_sequence,ESS,Annotation,H1_Canonical_Class,H2_Canonical_Class,Clusters,Epitope_ID,PDB
0,1d878900ca46e5b9792388b74a762b066f4e5ac8ceb5f8...,VESGGGLVQPGGSLRLSCAASGFNVSYSSIHWVRQAPGKGLEWVAY...,6az2A,1,6az2A,ARSYSTKLAMDY,93,True,H1-8-A,H2-8-A,6az2A,745537,6AZ2
1,51ed897d79dee863356c553cc1ae15f23a8ac434bb3b47...,VESGGGLVQPGGSLRLSCAASGFNVSYSSIHWVRQAPGKGLEWVAY...,5ueaA,1,5ueaA,ARSYSTKLAMDY,93,True,H1-8-A,H2-8-A,6az2A,745536,5UEA
2,e8692523aae84ec72e70a9cc6c04a3479af37976e6f858...,VESGGGLVQPGGSLRLSCAASGFNVSYYSIHWVRQAPGKGLEWVAS...,5ucbH,1,5ucbH,ARGYGWALDY,86,True,H1-8-A,H2-8-A,6ayzB,745539,5UCB
3,440fe070c6f266f845a8440ca877b6a114cdbedbce7b10...,VESGGGLVQPGGSLRLSCAASGFNVSYYSIHWVRQAPGKGLEWVAS...,6ayzB,1,6ayzB,ARGYGWALDY,86,True,H1-8-A,H2-8-A,6ayzB,745538,6AYZ
4,0a353f3dd246e85ba62ea11a5e939fd8849bfbc3a67e73...,VESGGGLVQPGGSLRLSCAASGFIVSSNYMSWVRQAPGKGLEWVSV...,5ifhH,1,4tsaH,AREAYGMDV,38,True,H1-8-A,H2-7-A,5ifhH,1075136,7BZ5


## SPACE2

In [12]:
# create integer mapping for unique strings
integer_mapping = {string: i for i, string in enumerate(space2["cluster_by_rmsd"].unique())}
# replace strings with integers 
space2["cluster_by_rmsd"] = space2["cluster_by_rmsd"].map(integer_mapping)

# extract ID from path string
space2["Clone_ID"] = space2["Clone_ID"].apply(rename_ID)
space2[:5]

,Clone_ID,cluster_by_length,cluster_by_rmsd
0,77aa8fa1f6202f9eda4c106c5692fd6d4969c7999e9f48...,13_10_17_11_8_11,0
1,b53df1fb98ab8045d2ed05445eecbb36f551f64d4eda74...,13_10_17_11_8_11,1
2,c46db1aef71b52fafc76df55c9a0d2e8e288854864ac6e...,13_10_17_11_8_11,2
3,f59ccf512f3cda19d92733080dee1f8d6ce7df8072e342...,13_10_17_11_8_11,3
4,f98730c169666dce8f080ec88eb79257ac7edfd8715cd5...,13_10_17_11_8_11,4


In [13]:
space2_antibody_pairs = space2.merge(epitope_clone_mapping, on="Clone_ID", how="inner")
print("Number of antibodies clustered by SPACE2:", len(space2_antibody_pairs))

Number of antibodies clustered by SPACE2: 54


# Cluster analysis

## SAAB+

### Number of clones and clusters

In [14]:
print("Number of clones without cluster assignment: %i (%.2f%%)" 
      %(len(saabplus_unclustered), len(saabplus_unclustered)/len(saabplus)*100))
print("Number of clones with cluster assignment: %i (%.2f%%)" 
      %(len(saabplus_clustered), len(saabplus_clustered)/len(saabplus)*100))

Number of clones without cluster assignment: 1302 (12.52%)
Number of clones with cluster assignment: 9100 (87.48%)


In [15]:
print("Number of clones:", len(saabplus_clustered["Protein_Seq"].unique()))
print("Number of clusters:", len(saabplus_clustered["Clusters"].unique()))
print("Average cluster size: %.3f" %(np.mean(saabplus_clustered["Clusters"].value_counts())))

Number of clones: 9100
Number of clusters: 973
Average cluster size: 9.353


### Multi-occupancy clusters

In [16]:
clone_count = saabplus_clustered["Clusters"].value_counts()
single_clone_clusters = clone_count[clone_count == 1].index
multi_clone_clusters = clone_count[clone_count > 1].index

In [17]:
print("Number of single-clone clusters:", sum(clone_count == 1))
print("Number of multi-clone clusters:", sum(clone_count > 1))
print("Average multi-clone cluster size: %.3f" \
    %(np.mean(saabplus_clustered[saabplus_clustered["Clusters"].isin(multi_clone_clusters)]["Clusters"].value_counts())))

Number of single-clone clusters: 210
Number of multi-clone clusters: 763
Average multi-clone cluster size: 11.651


### Save dataframe

In [18]:
saabplus_antibody_pairs.to_csv(os.path.dirname(os.getcwd()) + "/data/antibody_pairs/Antibody_pairs_clustered_SAAB+.csv",
    index=False)

## SPACE2

### Number of clones and clusters

In [19]:
print("Number of clones:", len(space2["Clone_ID"].unique()))
print("Number of clusters:", len(space2["cluster_by_rmsd"].unique()))
print("Average cluster size: %.3f" %(np.mean(space2["cluster_by_rmsd"].value_counts())))

Number of clones: 10489
Number of clusters: 8031
Average cluster size: 1.306


### Multi-occupancy clusters

In [20]:
clone_count = space2["cluster_by_rmsd"].value_counts()
single_clone_clusters = clone_count[clone_count == 1].index
multi_clone_clusters = clone_count[clone_count > 1].index

In [21]:
print("Number of single-clone clusters:", sum(clone_count == 1))
print("Number of multi-clone clusters:", sum(clone_count > 1))
print("Average multi-clone cluster size: %.3f" \
    %(np.mean(space2[space2["cluster_by_rmsd"].isin(multi_clone_clusters)]["cluster_by_rmsd"].value_counts())))

Number of single-clone clusters: 6579
Number of multi-clone clusters: 1452
Average multi-clone cluster size: 2.693


### Length-based clusters

In [22]:
print("Number of length-based clusters:", len(space2["cluster_by_length"].unique()))
print("Average length-based cluster size: %.3f" %(np.mean(space2["cluster_by_length"].value_counts())))

Number of length-based clusters: 1891
Average length-based cluster size: 5.547


In [23]:
clone_count = space2["cluster_by_length"].value_counts()
single_clone_clusters = clone_count[clone_count == 1].index
multi_clone_clusters = clone_count[clone_count > 1].index
print("Number of length-based single-clone clusters: %i (%.2f%%)" 
      %((sum(clone_count == 1)), (sum(clone_count == 1))/len(space2["cluster_by_length"].unique())*100))
print("Number of length-based multi-clone clusters: %i (%.2f%%)" 
      %((sum(clone_count > 1)), (sum(clone_count > 1))/len(space2["cluster_by_length"].unique())*100))

Number of length-based single-clone clusters: 808 (42.73%)
Number of length-based multi-clone clusters: 1083 (57.27%)


### Save dataframe

In [24]:
space2_antibody_pairs.to_csv(os.path.dirname(os.getcwd()) + "/data/antibody_pairs/Antibody_pairs_clustered_SPACE2.csv",
    index=False)

# Cluster comparison

In [25]:
cluster_comparison = antibody_pairs[["Epitope_ID_A", "Epitope_ID_B"]]

In [26]:
cluster_comparison["Clone_ID_A"] = cluster_comparison["Epitope_ID_A"].apply(add_clone_id, args=[epitope_clone_mapping])
cluster_comparison["Clone_ID_B"] = cluster_comparison["Epitope_ID_B"].apply(add_clone_id, args=[epitope_clone_mapping])
cluster_comparison.dropna(subset=["Clone_ID_A", "Clone_ID_B"], inplace=True)
print("Number of antibody pairs with assigned clone IDs:", len(cluster_comparison))
cluster_comparison[:5]

Number of antibody pairs with assigned clone IDs: 213


,Epitope_ID_A,Epitope_ID_B,Clone_ID_A,Clone_ID_B
0,164067,164069,bf226286707d82bda728cbef30375ccd358dc00fd3d175...,3302c6ccef79e396bf79043b81511f50c70bd7c476e560...
1,164078,164079,ad5b550aabf130663ecc53fb708035fdb1ac5c34e1096b...,6503ec1e63c6319e7adc1227f4302ed2d618a15dbc81c1...
2,969166,969167,2c6b76f8abeb2ae2cdba29efdd2096543d816cb5409b99...,9501c71df16e2728f9d63f1c999700760b7f43d20db86c...
3,969166,969168,2c6b76f8abeb2ae2cdba29efdd2096543d816cb5409b99...,24b1f7f80b240d6eb281a80900f0f2637f1aa8164a4060...
4,969167,969168,9501c71df16e2728f9d63f1c999700760b7f43d20db86c...,24b1f7f80b240d6eb281a80900f0f2637f1aa8164a4060...


## IGX-Cluster

In [27]:
cluster_comparison["IGX_A"] = cluster_comparison["Clone_ID_A"].apply(add_cluster, args=[cluster, "Unique Clone Id", 
    "Unique Cluster Id"])
cluster_comparison["IGX_B"] = cluster_comparison["Clone_ID_B"].apply(add_cluster, args=[cluster, "Unique Clone Id", 
    "Unique Cluster Id"])
cluster_comparison["IGX_clustered"] = np.where(cluster_comparison["IGX_A"] == cluster_comparison["IGX_B"], True, False)

In [28]:
cluster_comparison["IGX_clustered"].value_counts()

IGX_clustered
False    189
True      24
Name: count, dtype: int64

## SAAB+

In [29]:
cluster_comparison["SAABplus_A"] = cluster_comparison["Clone_ID_A"].apply(add_cluster, 
    args=[saabplus_clustered_clones, "Clone_ID", "Clusters"])
cluster_comparison["SAABplus_B"] = cluster_comparison["Clone_ID_B"].apply(add_cluster, 
    args=[saabplus_clustered_clones, "Clone_ID", "Clusters"])
cluster_comparison["SAABplus_clustered"] = np.where(cluster_comparison["SAABplus_A"] == cluster_comparison["SAABplus_B"], 
    True, False)

In [30]:
cluster_comparison["SAABplus_clustered"].value_counts()

SAABplus_clustered
False    197
True      16
Name: count, dtype: int64

## SPACE2

In [31]:
# add assigned cluster
cluster_comparison["SPACE2_A"] = cluster_comparison["Clone_ID_A"].apply(add_cluster, 
    args=[space2, "Clone_ID", "cluster_by_rmsd"])
cluster_comparison["SPACE2_B"] = cluster_comparison["Clone_ID_B"].apply(add_cluster, 
    args=[space2, "Clone_ID", "cluster_by_rmsd"])
# add length cluster
cluster_comparison["SPACE2_length_A"] = cluster_comparison["Clone_ID_A"].apply(add_cluster, 
    args=[space2, "Clone_ID", "cluster_by_length"])
cluster_comparison["SPACE2_length_B"] = cluster_comparison["Clone_ID_B"].apply(add_cluster, 
    args=[space2, "Clone_ID", "cluster_by_length"])

cluster_comparison["SPACE2_clustered"] = np.where(cluster_comparison["SPACE2_A"] == cluster_comparison["SPACE2_B"], 
    True, False)
cluster_comparison["SPACE2_clustered_length"] = np.where(cluster_comparison["SPACE2_length_A"] == \
    cluster_comparison["SPACE2_length_B"], True, False)

In [32]:
cluster_comparison["SPACE2_clustered"].value_counts()

SPACE2_clustered
False    199
True      14
Name: count, dtype: int64

In [33]:
cluster_comparison["SPACE2_clustered_length"].value_counts()

SPACE2_clustered_length
False    193
True      20
Name: count, dtype: int64

In [34]:
missed_pairs_space2 = cluster_comparison[(cluster_comparison["SPACE2_clustered_length"] == True) & \
    (cluster_comparison["SPACE2_clustered"] == False)]
missed_pairs_space2

,Epitope_ID_A,Epitope_ID_B,Clone_ID_A,Clone_ID_B,IGX_A,IGX_B,IGX_clustered,SAABplus_A,SAABplus_B,SAABplus_clustered,SPACE2_A,SPACE2_B,SPACE2_length_A,SPACE2_length_B,SPACE2_clustered,SPACE2_clustered_length
39,1087266,2134977,11c8da005bc94af6e9f2d235d8c9626f830452fcd1dabd...,71a244ac0d18725ec622c3c6d0f3075c16a9d36e44442b...,950,950,True,3cleH,2dd8H,False,3346,3348,13_9_11_11_8_11,13_9_11_11_8_11,False,True
70,1310038,2217933,70c736799f0d567331d7539a710ddb27afc1e0d77ead98...,00d47bc9bca525bc3808c5ad45a2045c262dd44178f031...,2785,1610,False,3duuD,5mesH,False,7114,7115,13_9_12_12_8_8,13_9_12_12_8_8,False,True
86,1311247,1392553,e3061e76a9983257bed28e9f7d730e4a9d3e41fb6910a0...,8616e0b4db9848d3b17b1e078a2f1e8840ebc13c35c10a...,969,4539,False,5gs1K,5gzoH,False,4866,4869,13_9_12_11_8_9,13_9_12_11_8_9,False,True
165,1346822,2134977,c2f3ed6d6f691c2ec604dc041f5c242fb0c37511481e4d...,71a244ac0d18725ec622c3c6d0f3075c16a9d36e44442b...,950,950,True,2dd8H,2dd8H,True,3346,3348,13_9_11_11_8_11,13_9_11_11_8_11,False,True
175,1346823,2144729,1ae2fb81015e7a30715df39f8cd0ee89be1727651d7c4e...,2da9afba0e2ab0e36f3bf984161568eec7065781699ae7...,2773,6061,False,5bvjF,3vi3H,False,3918,3922,13_9_11_11_8_10,13_9_11_11_8_10,False,True
197,2060661,2060664,44e1f015023558e07807a2e3989fbda82d06f48c5ca468...,82191f407191e1dace0a2367e9fd1462f963b32952699f...,8951,8951,True,3ijyB,3ijyB,True,7080,7081,13_10_17_14_8_12,13_10_17_14_8_12,False,True


In [35]:
epitope_ids_missed_pairs_space2 = set(missed_pairs_space2["Epitope_ID_A"]).union(set(missed_pairs_space2["Epitope_ID_B"]))
epitope_clone_mapping[epitope_clone_mapping["Epitope_ID"].isin(epitope_ids_missed_pairs_space2)]

,Epitope_ID,PDB,Clone_ID
18,1087266,6XC2,11c8da005bc94af6e9f2d235d8c9626f830452fcd1dabd...
20,1310038,7JMO,70c736799f0d567331d7539a710ddb27afc1e0d77ead98...
21,1311247,7CHB,e3061e76a9983257bed28e9f7d730e4a9d3e41fb6910a0...
32,1346822,7BEI,c2f3ed6d6f691c2ec604dc041f5c242fb0c37511481e4d...
34,1346823,7BEM,1ae2fb81015e7a30715df39f8cd0ee89be1727651d7c4e...
36,1392553,7CJF,8616e0b4db9848d3b17b1e078a2f1e8840ebc13c35c10a...
41,2134977,7CHP,71a244ac0d18725ec622c3c6d0f3075c16a9d36e44442b...
45,2060661,7U2E,44e1f015023558e07807a2e3989fbda82d06f48c5ca468...
46,2060664,7U2D,82191f407191e1dace0a2367e9fd1462f963b32952699f...
49,2144729,7WBZ,2da9afba0e2ab0e36f3bf984161568eec7065781699ae7...


## Cluster summary

In [36]:
cluster_summary = cluster_comparison[["Epitope_ID_A", "Epitope_ID_B", "Clone_ID_A", "Clone_ID_B", "IGX_clustered", 
    "SPACE2_clustered", "SAABplus_clustered"]]
cluster_summary["Clustered"] = cluster_summary[["IGX_clustered", "SPACE2_clustered", "SAABplus_clustered"]].sum(axis=1)
cluster_summary["Clustered"].value_counts().sort_index()

Clustered
0    184
1     13
2      7
3      9
Name: count, dtype: int64

# Save data

In [37]:
cluster_comparison.to_csv(os.path.dirname(os.getcwd()) + "/data/antibody_pairs/Antibody_pairs_cluster_comparison.csv",
    index=False)